In [2]:
from pyspark.sql import SparkSession

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'

In [4]:
spark = SparkSession.builder.master("local[5]").appName("VarunAssessment5").getOrCreate()

:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/varunm15t38hedu/.ivy2/cache
The jars for the packages stored in: /home/varunm15t38hedu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-54f56c4c-3b93-49bc-880a-94799b90df30;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 978ms :: artifacts dl 81ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [6]:
sensor = spark.readStream.format("kafka").option('kafka.bootstrap.servers', 'master:9092').option('subscribe', 'varun_muralidhar').load()

In [7]:
data = sensor.selectExpr("CAST(value AS STRING)") \
    .withColumn("timestamp", split(col("value"), ",")[0]) \
    .withColumn("sensor", split(col("value"), ",")[1].cast("string")) \
    .withColumn("reading", split(col("value"), ",")[2].cast("double"))

In [8]:
avg_df = data.groupBy(window("timestamp", "1 minute"), col("sensor")) \
             .agg(avg("reading").alias("average_reading"))

In [ ]:
query = avg_df.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

25/06/20 05:48:11 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5364b363-4b9f-46f4-ae13-6fa6d76759a3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/06/20 05:50:55 WARN DataStreamer: Slow waitForAckedSeqno took 41135ms (threshold=30000ms). File being written: /tmp/temporary-5364b363-4b9f-46f4-ae13-6fa6d76759a3/state/0/0/_metadata/.schema.7d4fb645-9e08-48c2-bdfc-1a6b0c6ab35b.TID12.tmp, block: BP-1364978691-172.31.64.10-1677754292516:blk_1079746168_6009401, Write pipeline datanodes: [DatanodeInfoWithStorage[172.31.64.11:9866,DS-8af7bd92-c7a4-44fb-b361-00496b8456c3,DISK], DatanodeInfoWithStorage[172.31.64.12:9866,DS-c7b06f49-e29e-4fd7-8923-882da8ea1f7c,DISK], DatanodeInfoWithStorage[172.31.64.10:9866,DS-c5d3f703-fc05-45d5-9749-75d35b7c1674,DISK]].
25/06

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+--------+------------------+
|              window|  sensor|   average_reading|
+--------------------+--------+------------------+
|{2025-06-19 02:40...|sensor_A|             68.49|
|{2025-06-19 02:40...|sensor_B|             52.43|
|{2025-06-19 02:39...|sensor_C|             33.34|
|{2025-06-19 02:42...|sensor_C|54.885000000000005|
|{2025-06-19 02:41...|sensor_C|             56.53|
|{2025-06-19 02:41...|sensor_B|              43.3|
|{2025-06-19 02:39...|sensor_A|             31.79|
|{2025-06-19 02:38...|sensor_B|             49.33|
+--------------------+--------+------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+--------+------------------+
|              window|  sensor|   average_reading|
+--------------------+--------+------------------+
|{2025-06-19 03:13...|sensor_A|62.144999999999996|
|{2025-06-19 03:12...|sensor_B|             38.68|
|{2025-06-19 03:00...|sensor_B|             49.09|
|{2025-06-19 03:07...|sensor_A|             48.66|
|{2025-06-19 03:01...|sensor_C|             41.95|
|{2025-06-19 03:17...|sensor_A|53.364999999999995|
|{2025-06-19 02:47...|sensor_A|             72.26|
|{2025-06-19 02:40...|sensor_A|             68.49|
|{2025-06-19 03:18...|sensor_B|             49.02|
|{2025-06-19 03:11...|sensor_C|            58.705|
|{2025-06-19 03:16...|sensor_A|             52.31|
|{2025-06-19 03:18...|sensor_C|             54.87|
|{2025-06-19 02:56...|sensor_A|             44.14|
|{2025-06-19 02:46...|sensor_A|             53.37|
|{2025-06-19 02:52...|sensor_C|     

25/06/20 05:54:47 WARN DataStreamer: Slow waitForAckedSeqno took 33596ms (threshold=30000ms). File being written: /tmp/temporary-5364b363-4b9f-46f4-ae13-6fa6d76759a3/state/0/187/.3.delta.89f71cba-2ca8-4879-9b0c-1652683dfdc6.TID599.tmp, block: BP-1364978691-172.31.64.10-1677754292516:blk_1079750550_6013783, Write pipeline datanodes: [DatanodeInfoWithStorage[172.31.73.191:9866,DS-e2b7285d-a9a5-47f8-bcee-e91f66a4d588,DISK], DatanodeInfoWithStorage[172.31.64.14:9866,DS-f823cf84-06f8-4cdd-b6e9-29d13d0292ee,DISK], DatanodeInfoWithStorage[172.31.64.13:9866,DS-e595a26d-2fb4-4683-b8bf-ad5a6b69a54b,DISK]].
25/06/20 05:54:48 WARN DataStreamer: Slow waitForAckedSeqno took 33061ms (threshold=30000ms). File being written: /tmp/temporary-5364b363-4b9f-46f4-ae13-6fa6d76759a3/state/0/191/.3.delta.c8ac03b3-fdf4-4f51-a497-9a2d40a632cd.TID603.tmp, block: BP-1364978691-172.31.64.10-1677754292516:blk_1079750581_6013814, Write pipeline datanodes: [DatanodeInfoWithStorage[172.31.73.191:9866,DS-e2b7285d-a9a5-4

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+--------+------------------+
|              window|  sensor|   average_reading|
+--------------------+--------+------------------+
|{2025-06-19 03:13...|sensor_A|62.144999999999996|
|{2025-06-19 03:12...|sensor_B|             38.68|
|{2025-06-19 03:00...|sensor_B|             49.09|
|{2025-06-19 03:07...|sensor_A|             48.66|
|{2025-06-19 03:01...|sensor_C|             41.95|
|{2025-06-19 03:17...|sensor_A|53.364999999999995|
|{2025-06-19 02:47...|sensor_A|             72.26|
|{2025-06-19 02:40...|sensor_A|             68.49|
|{2025-06-19 03:18...|sensor_B|             49.02|
|{2025-06-19 03:11...|sensor_C|            58.705|
|{2025-06-19 03:16...|sensor_A|             52.31|
|{2025-06-19 03:24...|sensor_C|             50.73|
|{2025-06-19 03:18...|sensor_C|             54.87|
|{2025-06-19 03:24...|sensor_A|             47.07|
|{2025-06-19 02:56...|sensor_A|     